# Image Data Preprocessing

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image as pilImg
import os 
import cv2
from datetime import datetime
import matplotlib.pyplot as plt
from keras import optimizers
import itertools
from datetime import datetime
from collections import Counter
from prettytable import PrettyTable

In [ ]:
!wget 'https://www.dropbox.com/s/y0jjlw4gvju9llf/parser.txt'

--2023-01-10 13:17:02--  https://www.dropbox.com/s/y0jjlw4gvju9llf/parser.txt?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/y0jjlw4gvju9llf/parser.txt [following]
--2023-01-10 13:17:02--  https://www.dropbox.com/s/raw/y0jjlw4gvju9llf/parser.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc02dc10a5e0c58bd7bbe6272367.dl.dropboxusercontent.com/cd/0/inline/B0Ttjo5wiAR1vLR0YJgS7TnFKXt5Cr9NJR1qIFfnaPnuurYOD12mKTKbleZF3j-iIbKzQjK5c9x7JxIAjgJSLglhC9oY-xV5UfSG2U9upw0aWF1SxRNYebSvNFNLkJexaqBuhU1pwwVzNyxOkOf34p3Au7KvnoBYSBE_32gwX_k9AQ/file# [following]
--2023-01-10 13:17:02--  https://uc02dc10a5e0c58bd7bbe6272367.dl.dropboxusercontent.com/cd/0/inline/B0Ttjo5wiAR1vLR0YJgS7TnFKXt5Cr9NJR1qIFfnaPnuurYOD12mKTKbleZF3j-iIbKzQjK5c9x7JxIAj

In [ ]:
!wget 'https://www.dropbox.com/s/0qtt1j8zfbxowet/words2.zip'

--2023-01-10 13:17:57--  https://www.dropbox.com/s/0qtt1j8zfbxowet/words2.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/0qtt1j8zfbxowet/words2.zip [following]
--2023-01-10 13:17:57--  https://www.dropbox.com/s/raw/0qtt1j8zfbxowet/words2.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc572f72c2b36054b33d5dbcf3e4.dl.dropboxusercontent.com/cd/0/inline/B0QbPKp8nJf92ZqKrf8cBawi3xE2NNWKri3LU6NRy5qi9lk-E9Yzhrhlim9_ySh5Dd305z0ZHufLKPF4dKFg5ug9KdmvwZ2gQGQwsjTS5k2ArMb6V63Rg8jt0oJI81THJhWRLhru_SplNKdugyrcjlczr69amSy_7ihb7SKTbBZgpQ/file# [following]
--2023-01-10 13:17:57--  https://uc572f72c2b36054b33d5dbcf3e4.dl.dropboxusercontent.com/cd/0/inline/B0QbPKp8nJf92ZqKrf8cBawi3xE2NNWKri3LU6NRy5qi9lk-E9Yzhrhlim9_ySh5Dd305z0ZHufLKPF4dKFg5u

In [ ]:
!unzip '/content/words2.zip'

In [ ]:
directory_path = '/content/words2/'
#/content/words2/a01/a01-000u/a01-000u-00-00.png
df2 = pd.read_csv("/content/parser.txt", sep=" ", header=None, 
                 names=["ImageId", '0','1','2','3','4','5','6','Labels'])
path =df2['ImageId'].str.split("-", n = 3, expand = True)
df2['ImageName']=directory_path+path[0]+'/'+path[0]+'-'+path[1]+'/'+df2['ImageId']+'.png'
df2['folder']=path[0]
df2 =df2[df2['folder'] =='a01']
dataFrame_ = df2[['ImageName','Labels']]

In [ ]:
from sklearn.model_selection import train_test_split
df, val_data = train_test_split(dataFrame_, test_size=0.2)

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.2)

In [ ]:
train_data.to_csv('Train_data.csv')
val_data.to_csv('Validation_data.csv')
test_data.to_csv('Test_data.csv')

In [ ]:
train_data=pd.read_csv('Train_data.csv')
train_data.drop(['Unnamed: 0'],axis=1,inplace=True)
train_files=train_data['ImageName'].values
train_data.to_csv('Train_Final.csv')

In [ ]:
val_data=pd.read_csv('Validation_data.csv')
val_data.drop(['Unnamed: 0'],axis=1,inplace=True)
val_files=val_data['ImageName'].values
val_data.to_csv('Validation_Final.csv')

In [ ]:
test_data=pd.read_csv('Test_data.csv')
test_data.drop(['Unnamed: 0'],axis=1,inplace=True)
test_files=test_data['ImageName'].values
test_data.to_csv('Test_Final.csv')

In [ ]:
train_data=pd.read_csv('Train_Final.csv')
val_data=pd.read_csv('Validation_Final.csv')
test_data=pd.read_csv('Test_Final.csv')

## 8. Utility Functions

In [ ]:
import keras
import random
from keras import backend as K
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Letters present in the Label Text
letters= '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [ ]:
img_h=32
img_w=170
img_c=1
num_classes=len(letters)+1
batch_size=64
max_length=15 

In [ ]:
def encode_words_labels(word):
    label_lst=[]
    for char in word:
        label_lst.append(letters.find(char)) 
    return label_lst

In [ ]:
def words_from_labels(labels):
    txt=[]
    for ele in labels:
        if ele == len(letters): 
            txt.append("")
        else:
            txt.append(letters[ele])
    return "".join(txt)

In [ ]:
def ctc_loss_function(args):

    y_pred, y_true, input_length, label_length = args 
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(y_true, y_pred, input_length, label_length)   

## 9. Data Generation

In [ ]:

class DataGenerator(keras.callbacks.Callback):
    def __init__(self, img_dirpath, img_w, img_h,
                 batch_size,n,output_labels,max_text_len=15):
        self.img_h = img_h                    
        self.img_w = img_w                   
        self.batch_size = batch_size         
        self.max_text_len = max_text_len    
        self.n=n
        self.img_dir = img_dirpath[:self.n]   
        self.indexes = list(range(self.n))   
        self.cur_index = 0                   
        self.imgs = np.zeros((self.n, self.img_h, self.img_w))
        self.texts =  output_labels[:self.n]                 

   
    def build_data(self):
        print(self.n, " Image Loading start...")
        for i, img_file in enumerate(self.img_dir):
            img = cv2.imread(img_file)
            img = img[:,:,1]                              
            img = cv2.resize(img, (self.img_w, self.img_h))
            img = img /255
            self.imgs[i, :, :]= img
            if i%10000==0:
                print("Loaded Images: ",i)
           
        print("Number of Texts matches with Total Number of Images :",len(self.texts) == self.n)
        print(self.n, " Image Loading finish...")


    def next_data(self): 
        """
        Returns image and text data pointed by the current index
        """
        self.cur_index += 1
        if self.cur_index >= self.n:
            self.cur_index = 0
            random.shuffle(self.indexes)
        return self.imgs[self.indexes[self.cur_index]], self.texts[self.indexes[self.cur_index]]

    def next_batch(self):
        """
        Creates a batch of images images and text data equal to the batch_size,
        computes the parameters needed for CTC and returns the inputs to the Model
        """
        while True:
            X_data = np.ones([self.batch_size, self.img_w, self.img_h, 1])  
            Y_data = np.ones([self.batch_size, self.max_text_len])* -1       
            input_length = np.ones((self.batch_size, 1)) * 40
            label_length = np.zeros((self.batch_size, 1))                   
            source_str=[]                                                  
            for i in range(self.batch_size):
                img, text = self.next_data() 
                img=img.T
                img = np.expand_dims(img, -1) 
                X_data[i] = img
                label=encode_words_labels(text) 
                lbl_len=len(label)
                Y_data[i,0:lbl_len] = label 
                label_length[i] = len(label)
                source_str.append(text) #
            inputs = {
                'img_input': X_data,  
                'ground_truth_labels': Y_data,  
                'input_length': input_length,  
                'label_length': label_length,
                'source_str': source_str 
            }
            outputs = {'ctc': np.zeros([self.batch_size])}  
            yield (inputs, outputs) 

## 12. Model Implementation

In [ ]:
from keras.layers import Input, Conv2D, MaxPool2D, Dense,MaxPooling2D
from keras.layers import AveragePooling2D, Flatten, Activation, Bidirectional
from keras.layers import BatchNormalization, Dropout
from keras.layers import Concatenate, Add, Multiply, Lambda
from keras.layers import UpSampling2D, Reshape
from tensorflow.keras.layers import add,concatenate
from keras.layers import Reshape
from keras.models import Model
from keras.layers import LSTM,GRU
import tensorflow as tf

## 12.1. Model 1

**Model with Bi-Directional LSTM units and Adam Optimizer**

In [ ]:
def Image_text_recogniser_model_1(stage,drop_out_rate=0.35):
    """
    Builds the model by taking in the stage variable which specifes the stage,
    if the stage is training: model takes inputs required for computing ctc_batch_cost function
    else : model takes input as images which is used for prediction
    """
    
    if K.image_data_format() == 'channels_first':
        input_shape = (1, img_w, img_h)
    else:
        input_shape = (img_w, img_h, 1)
       
    model_input=Input(shape=input_shape,name='img_input',dtype='float32')

    # Convolution layer 
    model = Conv2D(64, (3, 3), padding='same', name='conv1', kernel_initializer='he_normal')(model_input) 
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = MaxPooling2D(pool_size=(2, 2), name='max1')(model) 

    model = Conv2D(128, (3, 3), padding='same', name='conv2', kernel_initializer='he_normal')(model) 
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = MaxPooling2D(pool_size=(2, 2), name='max2')(model) 

    model = Conv2D(256, (3, 3), padding='same', name='conv3', kernel_initializer='he_normal')(model) 
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Conv2D(256, (3, 3), padding='same', name='conv4', kernel_initializer='he_normal')(model)
    model=Dropout(drop_out_rate)(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = MaxPooling2D(pool_size=(1, 2), name='max3')(model)  

    model = Conv2D(512, (3, 3), padding='same', name='conv5', kernel_initializer='he_normal')(model) 
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Conv2D(512, (3, 3), padding='same', name='conv6')(model)
    model=Dropout(drop_out_rate)(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = MaxPooling2D(pool_size=(1, 2), name='max4')(model) 

    model = Conv2D(512, (2, 2), padding='same', kernel_initializer='he_normal', name='con7')(model)
    model=Dropout(0.25)(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)    

    # CNN to RNN
    model = Reshape(target_shape=((42, 1024)), name='reshape')(model)  
    model = Dense(64, activation='relu', kernel_initializer='he_normal', name='dense1')(model)  

    # RNN layer
    model=Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='he_normal'), merge_mode='sum')(model)
    model=Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='he_normal'), merge_mode='concat')(model)

    # transforms RNN output to character activations:
    model = Dense(num_classes, kernel_initializer='he_normal',name='dense2')(model) 
    y_pred = Activation('softmax', name='softmax')(model)

    
    labels = Input(name='ground_truth_labels', shape=[max_length], dtype='float32') 
    input_length = Input(name='input_length', shape=[1], dtype='int64') 
    label_length = Input(name='label_length', shape=[1], dtype='int64') 

    #CTC loss function
    loss_out = Lambda(ctc_loss_function, output_shape=(1,),name='ctc')([y_pred, labels, input_length, label_length]) #(None, 1)

    if stage=='train':
        return model_input,y_pred,Model(inputs=[model_input, labels, input_length, label_length], outputs=loss_out)
    else:
        return Model(inputs=[model_input], outputs=y_pred)        

In [ ]:
model_input,y_pred,img_text_recog=Image_text_recogniser_model_1('train')

In [ ]:
test_func = K.function([model_input], [y_pred])

In [ ]:
img_text_recog.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img_input (InputLayer)         [(None, 170, 32, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 conv1 (Conv2D)                 (None, 170, 32, 64)  640         ['img_input[0][0]']              
                                                                                                  
 batch_normalization (BatchNorm  (None, 170, 32, 64)  256        ['conv1[0][0]']                  
 alization)                                                                                       
                                                                                              

In [ ]:
def decode_batch(test_func, word_batch):
    """
    Takes the Batch of Predictions and decodes the Predictions by Best Path Decoding and Returns the Output
    """
    out = test_func([word_batch])[0] 
    ret = []
    for j in range(out.shape[0]):
        out_best = list(np.argmax(out[j, 2:], 1))
        out_best = [k for k, g in itertools.groupby(out_best)]
        outstr = words_from_labels(out_best)
        ret.append(outstr)
    return ret

In [ ]:
def accuracies(actual_labels,predicted_labels,is_train):
    """
    Takes a List of Actual Outputs, predicted Outputs and returns their accuracy and letter accuracy across
    all the labels in the list
    """
    accuracy=0
    letter_acc=0
    letter_cnt=0
    count=0
    for i in range(len(actual_labels)):
        predicted_output=predicted_labels[i]
        actual_output=actual_labels[i]
        count+=1
        for j in range(min(len(predicted_output),len(actual_output))):
            if predicted_output[j]==actual_output[j]:
                letter_acc+=1
        letter_cnt+=max(len(predicted_output),len(actual_output))
        if actual_output==predicted_output:
            accuracy+=1
    final_accuracy=np.round((accuracy/len(actual_labels))*100,2)
    final_letter_acc=np.round((letter_acc/letter_cnt)*100,2)
    return final_accuracy,final_letter_acc

**CallBacks**

In [ ]:
class VizCallback(keras.callbacks.Callback):
    """
    The Custom Callback created for printing the Accuracy and Letter Accuracy Metrics at the End of Each Epoch
    """

    def __init__(self, test_func, text_img_gen,is_train,acc_compute_batches):
        self.test_func = test_func
        self.text_img_gen = text_img_gen
        self.is_train=is_train                
        self.acc_batches=acc_compute_batches 

    def show_accuracy_metrics(self,num_batches):
        """
        Calculates the accuracy and letter accuracy for each batch of inputs, 
        and prints the avarage accuracy and letter accuracy across all the batches
        """
        accuracy=0
        letter_accuracy=0
        batches_cnt=num_batches
        while batches_cnt>0:
            word_batch = next(self.text_img_gen)[0]   #Gets the next batch from the Data generator
            decoded_res = decode_batch(self.test_func,word_batch['img_input'])
            actual_res=word_batch['source_str']
            acc,let_acc=accuracies(actual_res,decoded_res,self.is_train)
            accuracy+=acc
            letter_accuracy+=let_acc
            batches_cnt-=1
        accuracy=accuracy/num_batches
        letter_accuracy=letter_accuracy/num_batches
        if self.is_train:
            print("Train Average Accuracy of "+str(num_batches)+" Batches: ",np.round(accuracy,2)," %")
            print("Train Average Letter Accuracy of "+str(num_batches)+" Batches: ",np.round(letter_accuracy,2)," %")
        else:
            print("Validation Average Accuracy of "+str(num_batches)+" Batches: ",np.round(accuracy,2)," %")
            print("Validation Average Letter Accuracy of "+str(num_batches)+" Batches: ",np.round(letter_accuracy,2)," %")
            
        
    def on_epoch_end(self, epoch, logs={}):
        self.show_accuracy_metrics(self.acc_batches)
        

In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
early_stop=EarlyStopping(monitor='val_loss',patience=2,restore_best_weights=True)
model_chk_pt=ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', save_best_only=False,save_weights_only=True,verbose=0, mode='auto', period=2)

**Labels Loading**

In [ ]:
#Loading Train Data Labels
Train_labels=[str(x) for x in train_data['Labels'].values]

In [ ]:
train_paths=[str(x) for x in train_data['ImageName'].values]

In [ ]:
train_nan_cnt=0
train_nan_replaced=False
for i in range(len(Train_labels)):
    if Train_labels[i]=='nan':
        Train_labels[i]='NULL'
        train_nan_replaced=True
        train_nan_cnt+=1

In [ ]:
print('Was there any NULL values written as Nan in Train Data:',train_nan_replaced)
print('Train Nan count: ',train_nan_cnt)

Was there any NULL values written as Nan in Train Data: False
Train Nan count:  0


In [ ]:
#Loading Validation Data Labels
cv_labels=[str(x) for x in val_data['Labels'].values]

In [ ]:
val_path=[str(x) for x in val_data['ImageName'].values]

In [ ]:
val_nan_cnt=0
val_nan_replaced=False
for i in range(len(cv_labels)):
    if cv_labels[i]=='nan':
        cv_labels[i]='NULL'
        val_nan_replaced=True
        val_nan_cnt+=1

In [ ]:
print('Was there any NULL values written as Nan :',val_nan_replaced)
print('Validation Nan count: ',val_nan_cnt)

Was there any NULL values written as Nan : False
Validation Nan count:  0


**Instatiating Data Generator**

In [ ]:
train_gene=DataGenerator(train_paths,img_w, img_h,batch_size,200000,Train_labels)

In [ ]:
train_gene.build_data()

200000  Image Loading start...
Loaded Images:  0


TypeError: ignored

In [ ]:
train_num_batches=int(train_gene.n / batch_size)

In [ ]:
viz_cb_train = VizCallback( test_func, train_gene.next_batch(),True,train_num_batches)

In [ ]:
val_gen=DataGenerator(val_path,img_w, img_h,batch_size,12000,cv_labels)

In [ ]:
val_gen.build_data()

12000  Image Loading start...
Loaded Images:  0
Number of Texts matches with Total Number of Images : False
12000  Image Loading finish...


In [ ]:
val_num_batches=int(val_gen.n / batch_size)

In [ ]:
viz_cb_val = VizCallback( test_func, val_gen.next_batch(),False,val_num_batches)

**Defining Optimizer**

In [ ]:
adam=optimizers.Adam()

In [ ]:
img_text_recog.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=adam)

In [ ]:
import datetime
logdir = os.path.join("logs_127", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

### 12.1.1. Train Model 1

In [ ]:
img_text_recog.fit_generator(generator=train_gene.next_batch(),
                    steps_per_epoch=int(train_gene.n / batch_size),
                    epochs=20,
                    callbacks=[viz_cb_train,viz_cb_val,train_gene,val_gen,tensorboard_callback,early_stop,model_chk_pt],
                    validation_data=val_gen.next_batch(),
                    validation_steps=int(val_gen.n / batch_size))
#callbacks=[viz_cb_train,viz_cb_val,train_gene,val_gen,tensorboard_callback,early_stop,model_chk_pt],

Epoch 1/20


InvalidArgumentError: ignored

In [ ]:
img_text_recog.save('Best_Img_recog_LSTM_Adam_model_run_weights.h5')

In [ ]:
img_text_recog.save('Img_recog_LSTM_Adam_model_run_3.h5')

## 13. Model Output Predictions

### 13.1. Best Path Decoding 

In [ ]:
def decode_label(out):
    """
    Takes the predicted ouput matrix from the Model and returns the output text for the image
    """
    out_best = list(np.argmax(out[0,2:], axis=1))

    out_best = [k for k, g in itertools.groupby(out_best)]  # remove overlap value

    outstr=words_from_labels(out_best)
    return outstr

### 13.2. Test Output Prediction Function

In [ ]:
def test_data_output_Prediction(model,test_img_names,test_labels):
    start=datetime.now()
    accuracy=0
    letter_acc=0
    letter_cnt=0
    count=0
    letter_mis_match=[]
    for i in range(len(test_labels)):
        test_img=cv2.imread(test_img_names[i])
        test_img_resized=cv2.resize(test_img,(170,32))
        test_image=test_img_resized[:,:,1]
        test_image=test_image.T
        test_image=np.expand_dims(test_image,axis=-1)
        test_image=np.expand_dims(test_image, axis=0)
        test_image=test_image/255
        model_output=model.predict(test_image)
        predicted_output=decode_label(model_output)
        actual_output=test_labels[i]
        count+=1
        mis_match=0
        for j in range(min(len(predicted_output),len(actual_output))):
            if predicted_output[j]==actual_output[j]:
                letter_acc+=1
            else:
                mis_match+=1
        letter_cnt+=max(len(predicted_output),len(actual_output))
        letter_mis_match.append(mis_match)
        if actual_output==predicted_output:
            accuracy+=1
        if (count%1000)==0:
            print("Processed ",count," Images")
    print("Time Taken for Processing: ",datetime.now()-start)
    return accuracy,letter_acc,letter_cnt,letter_mis_match

### 13.3. Model 1 Predictions

In [ ]:
model=Image_text_recogniser_model_1('predict')

In [ ]:
model.load_weights('Final_LSTM_Model_Best_Weights/Best_Img_recog_LSTM_Adam_model_run_weights.h5')

**Synth Text Validation Data Prediction**

In [ ]:
val_img_names=val_data['ImageName'].values
val_labels=val_data['Labels'].values

In [ ]:
synth_val_accuracy,synth_val_letter_acc,synth_val_letter_cnt,synth_val_mis_match=test_data_output_Prediction(model,val_img_names,val_labels)

In [ ]:
print("Model Output Accuracy: ",(synth_val_accuracy/len(val_labels))*100, " %")
print("Model Output Letter Accuracy: ",(synth_val_letter_acc/synth_val_letter_cnt)*100, " %")

In [ ]:
model_1_val_mis_match_dict=Counter(synth_val_mis_match)

**Model 1 Validation Data Prediction Analysis upto 4 Character Mis-Matches**

In [ ]:
mis_match_cnts_1=[]
for i in range(5):
    mis_match_cnts_1.append(model_1_val_mis_match_dict[i])

In [ ]:
def mis_match_character_analysis_plot(mis_match_counts,num_values):
    """
    Takes mis-match counts upto 4 characters of the predicted output, total number of values and
    plots the percentage of number of mis-match characters between predicted and actual labels
    """
    plt.figure(figsize=(10,6))
    indices=np.arange(len(mis_match_counts))
    counts=np.array(mis_match_counts)
    percent=(counts/num_values)*100
    plt.bar(indices,percent)
    plt.xlabel('Number of Mis-Match Characters',fontsize=10)
    plt.ylabel('Percentages',fontsize=10)
    plt.title('Percentages of Number of Mis-Match Characters',fontsize=12)
    plt.xticks(indices,indices)
    plt.show()
    for i in range(len(indices)):
        print(i," Mis-Match Characters Percentage: ",np.round(percent[i],2)," %")

**Synth Text Test Data**

In [ ]:
test_img_names=test_data['ImageName'].values
test_labels=test_data['Labels'].values

In [ ]:
synth_test_accuracy,synth_test_letter_acc,synth_test_letter_cnt,synth_test_mis_match=test_data_output_Prediction(model,test_img_names,test_labels)

In [ ]:
print("Model Output Accuracy: ",(synth_test_accuracy/len(test_labels))*100, " %")
print("Model Output Letter Accuracy: ",(synth_test_letter_acc/synth_test_letter_cnt)*100, " %")

Model Output Accuracy:  87.08666666666667  %
Model Output Letter Accuracy:  94.48166697375254  %


In [ ]:
model_1_test_mis_match_dict=Counter(synth_test_mis_match)

**Model 1 Test Data Prediction Analysis upto 4 Character Mis-Matches**

In [ ]:
mis_match_cnts_2=[]
for i in range(5):
    mis_match_cnts_2.append(model_1_test_mis_match_dict[i])

In [ ]:
mis_match_character_analysis_plot(mis_match_cnts_2,15000)

## 14. Results

1. **Model 1:** Model 1 Architecture consists of Convulution Layers, LSTM Units for RNN and uses Adam Optimizer
2. **Val:** Denotes Synth Text Validation Data containing 12000 Images
3. **Test:** Denotes Synth Text Test Data containing 15000 Images
4. **0 MM(%):** Denotes % of points out of total points which have 0 character mis-match between Predicted and Actual Labels
5. **1 MM(%):** Denotes % of points out of total points which have 1 character mis-match between Predicted and Actual Labels

In [ ]:
pt = PrettyTable()

pt.field_names = ["Model", "Data", "Accuracy","Letter Accuracy", "0 MM(%)", "1 MM (%)"]

pt.add_row(["Model 1", "Val", 86.47, 94.01, 86.58, 5.78])

pt.add_row(["Model 1", "Test", 87.98, 94.48, 87.19, 5.85])

print(pt)

+---------+------+----------+-----------------+---------+----------+
|  Model  | Data | Accuracy | Letter Accuracy | 0 MM(%) | 1 MM (%) |
+---------+------+----------+-----------------+---------+----------+
| Model 1 | Val  |  86.47   |      94.01      |  86.58  |   5.78   |
| Model 1 | Test |  87.98   |      94.48      |  87.19  |   5.85   |
| Model 2 | Val  |  81.86   |       92.1      |  82.06  |   7.32   |
| Model 2 | Test |  82.43   |      92.63      |  82.73  |   7.45   |
+---------+------+----------+-----------------+---------+----------+
